In [2]:
#!/usr/bin/python3
import math

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML, display
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    ControllabilityMatrix,
    DiagramBuilder,
    Linearize,
    LinearQuadraticRegulator,
    MeshcatVisualizer,
    Parser,
    Saturation,
    SceneGraph,
    Simulator,
    StartMeshcat,
    WrapToSystem,
    PointCloud,
    MeshcatPointCloudVisualizer,
    BaseField,
    Fields,
    Rgba
)
from pydrake.examples import (
    AcrobotGeometry,
    AcrobotInput,
    AcrobotPlant,
    AcrobotState,
    QuadrotorGeometry,
    QuadrotorPlant,
    StabilizingLQRController,
)
from pydrake.symbolic import Variable
from pydrake.systems.primitives import SymbolicVectorSystem
from pydrake.solvers import MathematicalProgram, Solve

import quadrotor_params as p

from trajectory_generator import TrajectoryGenerator
import waypoint_library
from differential_flatness import trajectory_to_state
import trajectory_controller

In [38]:
# waypoints = waypoint_library.waypoints_straight
waypoints = waypoint_library.waypoints_flip/2

headings = np.linspace(0.0, 2*np.pi, 2*waypoints.shape[1]-3).tolist()
for i in range(waypoints.shape[1]-3):
    del headings[-3 - i]
headings = np.array(headings)
headings = np.zeros(headings.shape)

traj_gen = TrajectoryGenerator(waypoints, headings)
traj_gen.solve()

# t = np.linspace(0, traj_gen.tf, 1000)
dt = .01
t = np.arange(0.0, traj_gen.tf, dt)

In [39]:
num_wp = waypoints.shape[1]
pts_per_edge = 20
edges_per_shape = 12
half_width = .2

waypoint_vis = PointCloud(new_size=num_wp*pts_per_edge*edges_per_shape, fields=Fields(BaseField.kXYZs))
wp_array = waypoint_vis.mutable_xyzs()

for i in range(waypoints.shape[1]):
    wp_offset = i*pts_per_edge*edges_per_shape
    j = 0
    for ii in [-half_width, half_width]:
        for jj in [-half_width, half_width]:
            for kk in np.linspace(-half_width, half_width, pts_per_edge):
                wp_array[:,wp_offset+j] = waypoints[:,i] + np.array([ii, jj, kk])
                j += 1
    for ii in [-half_width, half_width]:
        for jj in np.linspace(-half_width, half_width, pts_per_edge):
            for kk in [-half_width, half_width]:
                wp_array[:,wp_offset+j] = waypoints[:,i] + np.array([ii, jj, kk])
                j += 1
    for ii in np.linspace(-half_width, half_width, pts_per_edge):
        for jj in [-half_width, half_width]:
            for kk in [-half_width, half_width]:
                wp_array[:,wp_offset+j] = waypoints[:,i] + np.array([ii, jj, kk])
                j += 1
                

In [40]:
running_as_notebook = True
meshcat = StartMeshcat()
builder = DiagramBuilder()

plant = builder.AddSystem(QuadrotorPlant(
    m_arg=p.m_kg*5,
    L_arg=p.L_m,
    I_arg=p.J,
    kF_arg=p.kF,
    kM_arg=p.kM
))

point_cloud = PointCloud(new_size=len(t), fields=Fields(BaseField.kXYZs))
thing = point_cloud.mutable_xyzs()
for i, ti in enumerate(t):
    thing[:,i] = traj_gen.eval_trajectory(ti, 0)
meshcat.SetObject("/point_cloud/", point_cloud, point_size=.05, rgba=Rgba(1.0, 0.0, 0.5, 1.0))
meshcat.SetObject("/waypoints/", waypoint_vis, point_size=.05, rgba=Rgba(0.0, 0.5, 0.5, 1.0))

# Set up visualization in MeshCat
scene_graph = builder.AddSystem(SceneGraph())
QuadrotorGeometry.AddToBuilder(
    builder, plant.get_output_port(0), scene_graph
)
meshcat.Delete()
meshcat.ResetRenderMode()
meshcat.SetProperty("/Background", "visible", False)
MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
# end setup for visualization

diagram = builder.Build()

# Set up a simulator to run this diagram
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0 if running_as_notebook else 0.0)
context = simulator.get_mutable_context()
# print(context.GetInputPort())

INFO:drake:Meshcat listening for connections at http://localhost:7010


In [ ]:
# # Simulate
# for i in range(5):
#     context.SetTime(0.0)
#     # controller_context = diagram.GetMutableSubsystemContext(controller, context)
#     # controller_context.get_mutable_continuous_state_vector().SetFromVector(np.array([0.25, 0.25, 0.25, 0.25])*9.8)
#     # context.SetContinuousState(
#     #     0.5
#     #     * np.random.randn(
#     #         12,
#     #     )
#     # )
#     simulator.Initialize()
    
    
#     for ti in t:
#         simulator.AdvanceTo(ti)
#         context.SetContinuousState(
#             np.concatenate([traj_gen.eval_trajectory(ti, 0), np.zeros(2), traj_gen.eval_heading(ti, 0), np.zeros(6)])
#         )

In [41]:
# Simulate
for i in range(5):
    context.SetTime(0.0)
    simulator.Initialize()
    context.SetContinuousState(np.zeros(12))

    
    
    for ti in t[0:]:
        try:
            simulator.AdvanceTo(ti)
        except:
            simulator.Initialize()
        traj_matrix = traj_gen.eval_full_trajectory(ti)
        x = trajectory_to_state(traj_matrix, plant)
        # x_part = np.vstack([x[:6,:], np.zeros((3,1)), x[9:,:]])
        # x_part = np.vstack([x[:3,:], x[3:6,:], x[6:9, :], x[9:,:]])
        # x_fixed = np.vstack([x[:2,:], [-x[2,:]], x[3:8], [-x[8,:]], x[9:,:]])
        context.SetContinuousState(x)

# CONTROLLED

In [ ]:
running_as_notebook = True
meshcat = StartMeshcat()
builder = DiagramBuilder()

plant = builder.AddSystem(QuadrotorPlant(
    m_arg=p.m_kg,
    L_arg=p.L_m,
    I_arg=p.J,
    kF_arg=p.kF,
    kM_arg=p.kM
))

props = [Variable(f'p{i}') for i in range(4)]
controller = builder.AddSystem(SymbolicVectorSystem(state=props, dynamics=np.zeros(4), output=props))
builder.Connect(controller.get_output_port(0), plant.get_input_port(0))

# Set up visualization in MeshCat
scene_graph = builder.AddSystem(SceneGraph())
QuadrotorGeometry.AddToBuilder(
    builder, plant.get_output_port(0), scene_graph
)
meshcat.Delete()
meshcat.ResetRenderMode()
meshcat.SetProperty("/Background", "visible", False)
MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
# end setup for visualization

diagram = builder.Build()

# Set up a simulator to run this diagram
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0 if running_as_notebook else 0.0)
context = simulator.get_mutable_context()
# print(context.GetInputPort())

K_p = np.eye(3); K_v = np.eye(3)
K_R = .1*np.eye(3)
K_omega = .1*np.eye(3)

INFO:drake:Meshcat listening for connections at http://localhost:7015


In [ ]:
# Simulate
for i in range(5):
    context.SetTime(0.0)
    simulator.Initialize()

    controller_context = diagram.GetMutableSubsystemContext(controller, context)
    controller_state = controller_context.get_mutable_continuous_state_vector() #.SetFromVector(np.array([0.25, 0.25, 0.25, 0.25])*9.8)
    
    for ti in t[0:]:
        simulator.AdvanceTo(ti)
        traj_matrix = traj_gen.eval_full_trajectory(ti)
        # traj_state = trajectory_to_state(traj_matrix, plant)
        state = context.get_continuous_state_vector().CopyToVector().reshape((16,1))[:12,:]
        # state = np.array().reshape((12,1))
        print(state)
        controller_state.SetFromVector(trajectory_controller.update(traj_matrix, state, plant, K_p, K_v, K_R, K_omega))

        # context.SetContinuousState(x_part)

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[ 0.00000000e+00]
 [-2.19343606e-08]
 [ 7.20000000e-04]
 [ 1.41302571e-04]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [-1.14031174e-05]
 [ 1.43999999e-01]
 [ 2.82605142e-02]
 [ 0.00000000e+00]
 [ 0.00000000e+00]]
[[ 0.00000000e+00]
 [-4.44231638e-07]
 [ 2.87984880e-03]
 [ 5.57271035e-04]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [-9.05743532e-05]
 [ 2.87969754e-01]
 [ 5.49331786e-02]
 [ 0.00000000e+00]
 [ 0.00000000e+00]]
[[ 0.00000000e+00]
 [-2.24576615e-06]
 [ 6.47895049e-03]
 [ 1.23241415e-03]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [-3.02035145e-04]
 [ 4.31850554e-01]
 [ 8.00954435e-02]
 [ 0.00000000e+00]
 [ 0.00000000e+00]]
[[ 0.00000000e+00]
 [-7.08905165e-06]
 [ 1.15161374e-02]
 [ 2.15199706e-03]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [ 0.00000000e+00]
 [-7.06490654e-04]
 [ 5.75586756e-01]
 [ 1.03821140e-01]
 [ 0.00000000e+00]
 [ 0.00000000e+00]]
[[ 0.000000